In [1]:
%run ../../../ingestion/ingestion.ipynb
%run ../../../data\ quality/quality.ipynb
%run ../../../write/write.ipynb
%run ../../../alerts/webhook.ipynb

25/05/06 23:47:36 WARN Utils: Your hostname, lnb035652cps resolves to a loopback address: 127.0.1.1; using 192.168.1.172 instead (on interface wlp0s20f3)
25/05/06 23:47:36 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/06 23:47:37 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/05/06 23:47:39 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.
25/05/06 23:47:39 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [2]:
# Cria uma instância da classe WebhookTeams utilizando o arquivo de configuração localizado em ../../config/teams.json
webhook = WebhookTeams("../../../config/teams.json")

In [3]:
try:
    # Cria um leitor de CSV com delimitador de linha definido como ","
    reader = CSVReader(line_delimiter=",")
    
    # Executa a leitura do arquivo CSV e retorna um DataFrame
    df = client_code(reader, "../../../layer/bronze/saude/saude_leitos_mun_ano.csv")
except:
    # Em caso de erro durante a leitura, envia uma notificação via Teams
    webhook.send_teams_notification("Erro de leitura!")


In [4]:
df.show()

+--------+--------+------------+----------+----------+--------------+--------------+-----------------+
|cod_ibge|periodos|leitos_total|leitos_hab|leitos_sus|leitos_sus_hab|leitos_nao_sus|leitos_naosus_hab|
+--------+--------+------------+----------+----------+--------------+--------------+-----------------+
| 3500105|    2010|         308|       9,1|       271|             8|            37|              1,1|
| 3500204|    2010|           0|         0|         0|             0|             0|                0|
| 3500303|    2010|          76|       2,4|        49|           1,5|            27|              0,8|
| 3500402|    2010|           0|         0|         0|             0|             0|                0|
| 3500501|    2010|          42|       2,4|        30|           1,7|            12|              0,7|
| 3500550|    2010|           0|         0|         0|             0|             0|                0|
| 3500600|    2010|           0|         0|         0|             0|    

In [ ]:
try:
    # Cria um leitor de CSV com delimitador de linha definido como ","
    reader = ParquetReader()
    
    # Executa a leitura do arquivo CSV e retorna um DataFrame
    df_din = client_code(reader, "../../../layer/prata/dimensao/dimensao_ibge/*.parquet")
    df_din = df_din.select("cod_ibge", "municipios", "reg_administrativa")
except:
    # Em caso de erro durante a leitura, envia uma notificação via Teams
    webhook.send_teams_notification("Erro de leitura!")


In [6]:
# Instancia a classe de utilitários de qualidade de dados com o DataFrame original
dq = DataQualityUtils(df)

# Encadeia as transformações de qualidade de dados no DataFrame
df_tratado = (
    dq.remove_espacos_em_colunas()  # Remove espaços extras dos nomes das colunas
      .trim_strings()  # Remove espaços em branco no início e fim das strings de todas as colunas
      #.remover_caracteres_especiais(['localidades'])  # (Comentado) Remove caracteres especiais da coluna 'localidades'
      .remover_duplicatas()  # Remove registros duplicados do DataFrame
      .preencher_nulos({  # Preenche valores nulos com zero nas colunas especificadas
          'leitos_total': 0,
          'leitos_hab': 0,
          'leitos_sus': 0,
          'leitos_sus_hab': 0,
          'leitos_nao_sus': 0,
          'leitos_naosus_hab': 0
      })
      .converter_para_numerico(['leitos_total', 'leitos_hab',
                                'leitos_sus', 'leitos_sus_hab',
                                'leitos_nao_sus', 'leitos_naosus_hab'])
      .get_df()  # Retorna o DataFrame tratado
)


In [7]:
df_din.show()

+--------+--------------------+--------------------+
|cod_ibge|          municipios|       porte_pop2020|
+--------+--------------------+--------------------+
| 3500000|Estado sem especi...|Estado sem especi...|
| 3500105|          Adamantina|Mais de 20 mil at...|
| 3500204|              Adolfo|           Até 5 mil|
| 3500303|               Aguaí|Mais de 20 mil at...|
| 3500402|      Águas da Prata|Mais de 5 mil até...|
| 3500501|    Águas de Lindóia|Mais de 10 mil at...|
| 3500550|Águas de Santa Bá...|Mais de 5 mil até...|
| 3500600|  Águas de São Pedro|           Até 5 mil|
| 3500709|              Agudos|Mais de 20 mil at...|
| 3500758|            Alambari|Mais de 5 mil até...|
| 3500808|   Alfredo Marcondes|           Até 5 mil|
| 3500907|              Altair|           Até 5 mil|
| 3501004|         Altinópolis|Mais de 10 mil at...|
| 3501103|         Alto Alegre|           Até 5 mil|
| 3501152|            Alumínio|Mais de 10 mil at...|
| 3501202|    Álvares Florence|           Até 

In [8]:
df_tratado.show()

+--------+--------+------------+----------+----------+--------------+--------------+-----------------+
|cod_ibge|periodos|leitos_total|leitos_hab|leitos_sus|leitos_sus_hab|leitos_nao_sus|leitos_naosus_hab|
+--------+--------+------------+----------+----------+--------------+--------------+-----------------+
| 3509502|    2010|      2764.0|      NULL|    1407.0|          NULL|        1357.0|             NULL|
| 3530102|    2011|       126.0|      NULL|      96.0|          NULL|          30.0|             NULL|
| 3503406|    2012|        24.0|       3.0|      22.0|          NULL|           2.0|             NULL|
| 3545506|    2012|         0.0|       0.0|       0.0|           0.0|           0.0|              0.0|
| 3552908|    2012|         0.0|       0.0|       0.0|           0.0|           0.0|              0.0|
| 3504404|    2013|         0.0|       0.0|       0.0|           0.0|           0.0|              0.0|
| 3506102|    2013|       157.0|      NULL|     111.0|          NULL|    

In [ ]:
# Faz o relacionamento com a dimensão de códigos do IBGE
df_tratado = df_tratado.join(df_din, on='cod_ibge', how='left')

In [10]:
# Exibe as primeiras linhas do DataFrame tratado para visualização dos dados após as transformações
df_tratado.show()

+--------+--------+------------+----------+----------+--------------+--------------+-----------------+--------------------+--------------------+
|cod_ibge|periodos|leitos_total|leitos_hab|leitos_sus|leitos_sus_hab|leitos_nao_sus|leitos_naosus_hab|          municipios|       porte_pop2020|
+--------+--------+------------+----------+----------+--------------+--------------+-----------------+--------------------+--------------------+
| 3509502|    2010|      2764.0|      NULL|    1407.0|          NULL|        1357.0|             NULL|            Campinas|    Mais de 1 milhão|
| 3530102|    2011|       126.0|      NULL|      96.0|          NULL|          30.0|             NULL|        Mirandópolis|Mais de 20 mil at...|
| 3503406|    2012|        24.0|       3.0|      22.0|          NULL|           2.0|             NULL|             Arealva|Mais de 5 mil até...|
| 3545506|    2012|         0.0|       0.0|       0.0|           0.0|           0.0|              0.0|         Sandovalina|       

In [11]:
try:
    # Cria uma instância da classe responsável por exportar em formato Parquet
    exporter = ParquetExport()

    # Exporta o DataFrame para o caminho local especificado
    exporter.export(df_tratado, "../../../layer/prata/saude/saude_leitos/", destination="local")
except:
    # Em caso de erro durante a exportação, envia uma notificação via Teams
    webhook.send_teams_notification("Erro de escrita!")
